In [1]:
from cosmiclib import Compressor, Optimizer

In [8]:
compressor1 = Compressor('Compressor 1', qmin=0.5, qmax=1, wmin=0.5, wmax=1)
compressor2 = Compressor('Compressor 2', qmin=0.5, qmax=0.75, wmin=0.75, wmax=7/8)
my_optimizer = Optimizer(compressors=[compressor1, compressor2])
my_optimizer.setup_problem()
my_optimizer.find_opt(1.5)

Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 1 substitutions
Cgl0004I processed model has 4 rows, 2 columns (1 integer (1 of which binary)) and 8 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 10.000%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 0.5
Cbc0038I Relaxing continuous gives 0.5
Cbc0038I Before mini branch and bound, 1 integers at bound fixed and 1 continuous
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I After 0.00 seconds - Feasibility pump exiting with objective of 0.5 - took 0.00 seconds
Cbc0012I Integer solution of 0.5 found by feasibility pump after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 0.5, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Total time (CPU seconds):       0.00   (Wallclock seconds):     

([True, True], [0.75, 0.75])